## Requirements

In [ ]:
#! pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from itertools import combinations

In [ ]:
# Load the CSV files to compare
scn_FileName00= "scene_v1_501_cls0"
scn_FileName01= "cls0_scene_231027_unfiltered"
#scn_FileName02= "scene_curated_v0920"
df0 = pd.read_csv(f'../_output_scene/{scn_FileName00}.csv', usecols=[0, 1, 2])
df0['df'] = scn_FileName00  # Add a new column to indicate the source
df1 = pd.read_csv(f'../_output_scene/{scn_FileName01}.csv', usecols=[0, 1, 2])
df1['df'] = scn_FileName01  # Add a new column to indicate the source
# df2 = pd.read_csv(f'../_output_scene/{scn_FileName02}.csv')
# df2['df'] = scn_FileName02  # Add a new column to indicate the source

# concatenating dataframes into df
df = pd.concat([df0, df1], axis=0)
#df = pd.concat([df0, df1, df2], axis=0)

# 유사도 계산에 사용할 열 선택
selected_column = 'scn_sentence'
data = df[selected_column].values

## Similarity check

In [ ]:
# 텍스트 데이터를 벡터로 변환
vectorizer = CountVectorizer().fit_transform(data)
vectors = vectorizer.toarray()

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(vectors)

# 코사인 유사도가 비슷한 행 추출
similar_rows = []
threshold = 0.7  # 유사도 임계값 설정

for i, row in enumerate(cosine_similarities):
    similar_indices = [j for j, similarity in enumerate(row) if i != j and similarity > threshold]
    if similar_indices:
        similar_indices.append(i)  # 자기 자신도 포함
        similar_rows.append(similar_indices)

# 중복된 인덱스 제거
unique_similar_rows = []
for row_indices in similar_rows:
    if not any(set(row_indices).issubset(set(unique_indices)) for unique_indices in unique_similar_rows):
        unique_similar_rows.append(row_indices)
        
# 추출된 결과 출력
for i, row_indices in enumerate(unique_similar_rows):
    if i >= 4:  # 1~4개의 행까지만 출력
        break
    print(f"Group {i + 1}:")
    for idx in row_indices:
        print(df.iloc[idx])
        print("-" * 20)
    print("=" * 40)

## Saving as csv file

In [ ]:
# Display and save the results
output_csv_data = []

# Extract and display results
for i, row_indices in enumerate(unique_similar_rows):
    
    # 1~4개의 행까지만 출력
    if i < 4:
        print(f"Group {i + 1}:")
        for idx in row_indices:
            print(df.iloc[idx])
            print("-" * 20)
        print("=" * 40)
    
    group_label = f"Group {i + 1}"  # 그룹 레이블 생성
    output_csv_data.append([group_label, ""])  # 그룹 레이블 추가

    for idx in row_indices:
        output_csv_data.append([group_label] + df.iloc[idx].tolist())

# Create a DataFrame from the results
output_df = pd.DataFrame(output_csv_data, columns=["Group", "scn_id", "scn_sentence", "scn_cls", "file_name"])  

# Save as CSV
output_df.to_csv(f'sim_check_{scn_FileName00}_{scn_FileName01}.csv', index=False)
#output_df.to_csv(f'sim_check_{scn_FileName00}_{scn_FileName01}_{scn_FileName02}.csv', index=False)

print("The results have been saved to a CSV file.")
